# NLP Projects: 03 - Name Entity Recognition with Russian Drug Reaction Corpus

In [1]:
!pip install Partial State

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for Partial: filename=partial-1.0-py3-none-any.whl size=1888 sha256=e1dbbc56440d2bb58d233ad42b611156dcc49c880189226ced4304222431edcb
  Stored in directory: /home/jovyan/.cache/pip/wheels/9c/35/5e/9733a2210f6daf251fa7a12a01ae7b44225c8adc2d8d68f7c0
  Created wheel for State: filename=state-0.1.1.dev0-py3-none-any.whl size=1991 sha256=c88b16b4e18d70b944f653def64d196de4add01269d9a7b9ef8d57fe63bd05d7
  Stored in directory: /home/jovyan/.cache/pip/wheels/05/15/c1/eb3b661ad468c379f6b43a61dc0af9a72b601d4ae643080c91
Successfully built Partial State


In [2]:
!pip install --upgrade accelerate
!pip install --upgrade transformers

In [3]:
! pip install datasets transformers seqeval corus razdel -q

В этом блокноте происходит дообучение модели на задаче классификации отдельных слов, а именно, распознавание именованных сущностей (aka named entity recognition, aka NER). Беру датасет медицинских сущностей, но в целом пайплайн достаточно универсальный для задачи NER.

Для скорости я беру маленький BERT для русского языка [rubert-tiny](https://huggingface.co/cointegrated/rubert-tiny2); если взять другую, более крупную BERT-подобную модель, качество NER может быть выше, но и время обучения и работы будет дольше


In [162]:
model_checkpoint = "cointegrated/rubert-tiny2"
batch_size = 16

## Loading the dataset

Датасет текстов [Russian Drug Reaction Corpus](https://github.com/cimm-kzn/RuDReC): размеченный корпус русскоязычных отзывов на лекарства. Использую corus для загрузки датасета

In [163]:
from datasets import load_dataset, load_metric

In [164]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2023-10-23 00:38:59--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2023-10-23 00:39:00--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json.1’

rudrec_annotated.js 100%[===================>]   1.69M  7.86MB/s    in 0.2s    

2023-10-23 00:39:00 (7.86 MB/s) - ‘rudrec_annotated.json.1’ saved [1773014/1773014]



In [165]:
from corus import load_rudrec
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


Пример документа:

In [171]:
drugs[0]

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

Какие сущности в тексте есть: лекарства, форма лекарств, класс лекарств, показания к применению, побочки, и прочие болезни/симптомы.

https://arxiv.org/abs/2004.03659

* **DRUGNAME** Mentions of the brand name of a drug or product
ingredients/active compounds.
* **DRUGCLASS** Mentions of drug classes such as anti-inflammatory or
cardiovascular.
* **DRUGFORM** Mentions of routes of administration such as tablet
or liquid that describe the physical form in which
medication will be delivered into patient’s organism.
* **DI** Any indication/symptom that specifies the reason for
taking/prescribing the drug.
* **ADR** Mentions of untoward medical events that occur as a
consequence of drug intake and are not associated with
treated symptoms.
* **FINDING** Any DI or ADR that was not directly experienced by the
reporting patient or his/her family members, or related to
medical history/drug label, or any disease entities if the
annotator is not clear about type


In [172]:
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [173]:
drugs[0].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

Далее реализую функцию, перекладывающую разметку сущностей на уровень слов. Будем использовать [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))-нотацию, чтобы разделять несколько сущностей одного типа, идущих подряд.

In [174]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [175]:
print(extract_labels(drugs[0]))

{'tokens': ['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [176]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

Пример данных

In [177]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
3692,"[Мочить, плечо, запретили, .]","[O, O, O, O]"
184,"[Горло, как, было, красное, так, и, осталось, .]","[B-DI, I-DI, I-DI, I-DI, O, O, O, O]"
1621,"[Одной, упаковкой, не, обойтись, .]","[O, O, O, O, O]"


In [178]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Складываю данные в объект [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), нативный для huggingface.

In [180]:
from datasets import Dataset, DatasetDict

In [181]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3847
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 962
    })
})

## Preprocessing the data

In [183]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [184]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Так как наши тексты уже разбиты на слова, ставим атрибут `is_split_into_words=True`:

In [185]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 7440, 996, 6301, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Даже если наши данные уже были разделены на слова, каждое из этих слов может быть снова разделено токенизатором на под-токены. Возьмем пример:

In [187]:
example = ner_train[5]
print(example["tokens"])

['Мы', 'поменяли', 'место', 'жительства', 'и', 'перевели', 'дочь', 'в', 'школу', ',', 'которая', 'находится', 'ближе', 'к', 'дому', '.']


In [188]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Мы', 'поменяли', 'место', 'жительства', 'и', 'перевели', 'дочь', 'в', 'школу', ',', 'которая', 'находится', 'ближе', 'к', 'дому', '.', '[SEP]']


Чтобы перейти с уровня слов на уровень subword tokens, нужно ещё раз предобработать тексты.

In [189]:
len(example["tags"]), len(tokenized_input["input_ids"])

(16, 18)

In [190]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, None]


None здесь - это специальные токены начала и конца CLS и SEP. Заалайним токены между собой:

In [191]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

18 18


Здесь я устанавливаю значения всех специальных токенов как -100 (индекс, который игнорируется PyTorch), а метки всех остальных токенов на значение слова, из которого они происходят.

Внизу реализую функцию, которая будет токенайзить наши тексты. Мы отправляем предложения в токенизатор с аргументом truncation=True (для обрезания текстов, размер которых превышает максимальный размер, разрешенный моделью) и is_split_into_words=True

In [193]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Специльные токены помечены как None. Я ставлю значение -100, которое автоматически игнорируется в loss функции.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [194]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 1041, 37038, 33265, 19106, 40305, 22018, 548, 22276, 320, 21538, 16, 47886, 548, 59614, 11137, 626, 56606, 700, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 1, -100, 7, 7, 7, 7, 0, 0, 0, 0, -100, -100, 0, -100, 0, -100]]}

In [32]:
ner_data['train'][22:23]

{'tokens': [['На',
   'следующее',
   'утро',
   'появились',
   'высыпания',
   'на',
   'лице',
   'и',
   'руках',
   ',',
   'похожие',
   'на',
   'комариные',
   'укусы',
   '.']],
 'tags': [['O',
   'O',
   'O',
   'O',
   'B-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']]}

Замапим эту функцию на все предложения в нашем корпусе данных текстов 

In [195]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

## Fine-tuning the model

Используем `AutoModelForTokenClassification` класс, и как с токенайзером загружаем модель и кэш с помощью `from_pretrained` мотода:

In [246]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are new

Чтобы создать `Trainer` нам нужно объявить [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) со всеми атрибутами для обучения и fine-tuning'a модели:

In [247]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

Будем использоать data collator чтобы заппадить все примеры до одинакового размера, чтобы наши батчи были равными:

In [248]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

Подгружаю [`seqeval`](https://github.com/chakki-works/seqeval) фрйемворк для подсчета метрик на предсказанных нашей моделью значениях меток классов токенов:

In [249]:
metric = load_metric("seqeval")

Метрика берет list лейблов для предсказаний и таргетов. Попробуем указать `predictions` и `references` одинаковыми:

In [250]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'DI': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Drugform': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

Функция `compute_metrics` выполняет постобработку результата `Trainer.evaluate` (который представляет собой namedtuple, содержащий прогнозы и таргеты) перед применением подсчета метрик `metric.compute`:

In [251]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Объявим `Trainer`, куда передадим нашу модель, сплиты и токеназйер, с подсчетом метрик:

In [252]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [253]:
trainer.evaluate()

/home/jovyan/.imgenv-ars-100-2-80gb-0/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 2.5856988430023193,
 'eval_precision': 0.009246020398436755,
 'eval_recall': 0.10253699788583509,
 'eval_f1': 0.016962490163504415,
 'eval_accuracy': 0.09824924281598582,
 'eval_runtime': 1.0215,
 'eval_samples_per_second': 941.758,
 'eval_steps_per_second': 30.348}

#### 'eval_accuracy': 0.09824924281598582
Как видим по `eval_accuracy` - результаты плохие, явно можно лучше. Сделаем в начале обучения заморозку всех параметров в модели, кроме последнего слоя, и посмотрим, насколько хорошо она обучится.

In [254]:
for param in model.bert.parameters():
    param.requires_grad = False

In [255]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[-0.0028, -0.0077,  0.0072,  ...,  0.0136, -0.0096, -0.0042],
        [ 0.0055, -0.0254,  0.0016,  ..., -0.0128,  0.0304,  0.0035],
        [ 0.0530, -0.0260, -0.0194,  ..., -0.0048, -0.0108,  0.0080],
        ...,
        [-0.0274,  0.0244, -0.0038,  ...,  0.0081,  0.0057,  0.0079],
        [ 0.0306, -0.0392, -0.0050,  ..., -0.0200, -0.0196, -0.0021],
        [ 0.0127, -0.0098,  0.0271,  ...,  0.0346, -0.0143, -0.0129]],
       device='cuda:0', requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)


In [256]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [257]:
trainer.train()

/home/jovyan/.imgenv-ars-100-2-80gb-0/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.268893,0.013158,0.104651,0.023377,0.360346
2,No log,1.998460,0.019383,0.075053,0.030809,0.667504
3,No log,1.772536,0.032943,0.047569,0.038927,0.830021
4,No log,1.588418,0.051230,0.026427,0.034868,0.881436
5,2.031100,1.442692,0.056701,0.011628,0.019298,0.894733
6,2.031100,1.330649,0.061856,0.006342,0.011505,0.898574
7,2.031100,1.248439,0.098039,0.005285,0.010030,0.900052
8,2.031100,1.192462,0.161290,0.005285,0.010235,0.900717
9,1.351900,1.159981,0.200000,0.004228,0.008282,0.900864
10,1.351900,1.149274,0.222222,0.004228,0.008299,0.901012


TrainOutput(global_step=1210, training_loss=1.6027033151673877, metrics={'train_runtime': 52.846, 'train_samples_per_second': 727.964, 'train_steps_per_second': 22.897, 'total_flos': 29788281443220.0, 'train_loss': 1.6027033151673877, 'epoch': 10.0})

Модель недообучилась: похоже, что нужно обучить больше слоёв. Разморозим их все (но, воможно, более правильно было бы разморозить лишь несколько верхних), и поучимся ещё эпох 20.

In [258]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [259]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [260]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [261]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.388690,0.549020,0.029598,0.056169,0.903228
2,No log,0.325891,0.432039,0.282241,0.341432,0.918372
3,No log,0.294819,0.476383,0.373150,0.418494,0.924799
4,No log,0.275313,0.462601,0.424947,0.442975,0.926941
5,0.359800,0.260409,0.484581,0.465116,0.474649,0.930413
6,0.359800,0.248822,0.490967,0.488372,0.489666,0.932555
7,0.359800,0.240835,0.498477,0.519027,0.508545,0.934328
8,0.359800,0.232761,0.520790,0.529598,0.525157,0.936544
9,0.222300,0.227617,0.509785,0.550740,0.529472,0.937357
10,0.222300,0.222630,0.531936,0.563425,0.547228,0.939130


TrainOutput(global_step=2420, training_loss=0.2224559783935547, metrics={'train_runtime': 137.7917, 'train_samples_per_second': 558.379, 'train_steps_per_second': 17.563, 'total_flos': 59637835203600.0, 'train_loss': 0.2224559783935547, 'epoch': 20.0})

In [262]:
trainer.evaluate()

{'eval_loss': 0.2039673626422882,
 'eval_precision': 0.5643469971401335,
 'eval_recall': 0.6257928118393234,
 'eval_f1': 0.593483709273183,
 'eval_accuracy': 0.9453350077565191,
 'eval_runtime': 0.9588,
 'eval_samples_per_second': 1003.307,
 'eval_steps_per_second': 32.331,
 'epoch': 20.0}

#### 'eval_accuracy': 0.9453350077565191
Результаты на отложенной выборке уже лучше, сделаем `predict` для каждого вида лейбла класса и усредним значения:

In [263]:
p = trainer.predict(tokenized_datasets["test"])

predictions, labels, inputs = p.predictions, p.label_ids, tokenized_datasets["test"]['input_ids']
predictions = np.argmax(p.predictions, axis=2)

# используем только первый токен каждого слова для эвалюйта
true_predictions = []
true_labels = []
for prediction, label, tokens in zip(predictions, labels, inputs):
    true_predictions.append([])
    true_labels.append([])
    for (p, l, t) in zip(prediction, label, tokens):
        if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
            true_predictions[-1].append(label_list[p])
            true_labels[-1].append(label_list[l])

results = metric.compute(predictions=true_predictions, references=true_labels)
results

/home/jovyan/.imgenv-ars-100-2-80gb-0/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ADR': {'precision': 0.22641509433962265,
  'recall': 0.1610738255033557,
  'f1': 0.18823529411764703,
  'number': 149},
 'DI': {'precision': 0.37748344370860926,
  'recall': 0.6333333333333333,
  'f1': 0.4730290456431535,
  'number': 270},
 'Drugclass': {'precision': 0.8809523809523809,
  'recall': 0.5068493150684932,
  'f1': 0.6434782608695652,
  'number': 73},
 'Drugform': {'precision': 0.8532608695652174,
  'recall': 0.9127906976744186,
  'f1': 0.8820224719101123,
  'number': 172},
 'Drugname': {'precision': 0.7689393939393939,
  'recall': 0.8982300884955752,
  'f1': 0.8285714285714285,
  'number': 226},
 'Finding': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 56},
 'overall_precision': 0.5643469971401335,
 'overall_recall': 0.6257928118393234,
 'overall_f1': 0.593483709273183,
 'overall_accuracy': 0.9453350077565191}

Построим `confusion_matrix` и посмотрим на взаимосвязи между лейблами классов токенов:

In [264]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [265]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,12064,11,42,2,21,20,0,7,28,0,0,0,0
B-ADR,43,32,65,0,0,1,0,7,1,0,0,0,0
B-DI,58,3,189,2,2,8,0,1,7,0,0,0,0
B-Drugclass,8,0,21,37,1,6,0,0,0,0,0,0,0
B-Drugform,11,0,0,0,158,3,0,0,0,0,0,0,0
B-Drugname,5,0,2,0,2,217,0,0,0,0,0,0,0
B-Finding,13,5,36,0,0,1,0,0,1,0,0,0,0
I-ADR,65,11,12,0,0,0,0,42,39,0,0,0,0
I-DI,81,2,30,0,0,0,0,5,58,0,0,0,0
I-Drugclass,0,0,0,1,0,0,0,0,0,0,0,0,0


In [266]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

# Применение модели

In [267]:
import torch

In [268]:
# text = ' '.join(ner_test[4]['tokens'])  # 458 is example of a difficult text
text = "Совсем недавно началась весенняя аллергия, попробовал преобрести Зодак в аптеке, средсто очень хорошо помогает."
text

'Совсем недавно началась весенняя аллергия, попробовал преобрести Зодак в аптеке, средсто очень хорошо помогает.'

In [269]:
tokens = tokenizer(text, return_tensors='pt').to(model.device)
tokens

{'input_ids': tensor([[    2, 46015, 30697, 18746, 30356, 80093, 76809,    16, 63444, 33098,
         31564,   287,  7525,   865,   314, 57742,    16, 14276, 13151,  6003,
         14105, 30903,    18,     3]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [270]:
with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 24, 13])

In [271]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
indices

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [272]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
Совсем          O         
недавно         O         
началась        O         
весе            O         
##нняя          O         
аллергия        B-DI      
,               O         
попробовал      O         
преоб           O         
##рести         O         
З               B-Drugname
##ода           B-Drugname
##к             O         
в               O         
аптеке          O         
,               O         
сред            O         
##сто           O         
очень           O         
хорошо          O         
помогает        O         
.               O         
[SEP]           O         


Более простое применение модели: пайплайн от huggingface

In [273]:
from transformers import pipeline

In [274]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [275]:
print(text)
print(pipe(text))

Совсем недавно началась весенняя аллергия, попробовал преобрести Зодак в аптеке, средсто очень хорошо помогает.
[{'entity_group': 'DI', 'score': 0.27208838, 'word': 'аллергия', 'start': 33, 'end': 41}, {'entity_group': 'Drugname', 'score': 0.47578454, 'word': 'Зодак', 'start': 65, 'end': 70}]


Попробуем теперь замораживать до начала обучения не весь берт, а только первый его модуль, и учить в течении 30 эпох.

In [276]:
for num, param in enumerate(model.parameters()):
    if num==0:
        param.requires_grad = False
    else:
        param.requires_grad = True

In [277]:
# аргументы trainer
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

# trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/home/jovyan/.imgenv-ars-100-2-80gb-0/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/home/jovyan/.imgenv-ars-100-2-80gb-0/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.199797,0.570605,0.627907,0.597886,0.944522
2,No log,0.193937,0.595192,0.654334,0.623364,0.948881
3,No log,0.193286,0.569573,0.649049,0.606719,0.946886
4,No log,0.189820,0.594492,0.661734,0.626313,0.949546
5,0.150800,0.186987,0.619094,0.664905,0.641182,0.951097
6,0.150800,0.183669,0.610315,0.675476,0.641244,0.951614
7,0.150800,0.183759,0.606174,0.684989,0.643176,0.951392
8,0.150800,0.180566,0.602022,0.692389,0.644051,0.951392
9,0.131700,0.179735,0.595109,0.694503,0.640976,0.951392
10,0.131700,0.179088,0.608736,0.692389,0.647873,0.952648


TrainOutput(global_step=3630, training_loss=0.11716054622135215, metrics={'train_runtime': 204.0742, 'train_samples_per_second': 565.53, 'train_steps_per_second': 17.788, 'total_flos': 89283369306300.0, 'train_loss': 0.11716054622135215, 'epoch': 30.0})

In [278]:
trainer.evaluate()

{'eval_loss': 0.17131195962429047,
 'eval_precision': 0.6373831775700934,
 'eval_recall': 0.7209302325581395,
 'eval_f1': 0.6765873015873015,
 'eval_accuracy': 0.9549383172046982,
 'eval_runtime': 0.9836,
 'eval_samples_per_second': 978.089,
 'eval_steps_per_second': 31.518,
 'epoch': 30.0}

####  'eval_accuracy': 0.9549383172046982 - SOTA SCORE